In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
import joblib
import pandas as pd
import os

In [ ]:
directory='images'
filespath=[]
labels=[]

folds=os.listdir(directory)
for fold in folds:
    foldpath=os.path.join(directory,fold)
    filelist=os.listdir(foldpath)
    for file in filelist:
        fpath=os.path.join(foldpath, file)
        filespath.append(fpath)
        labels.append(fold)

Fseries=pd.Series(filespath,name='Gestures')
Lseries=pd.Series(labels,name='Labels')

df=pd.concat([Fseries,Lseries],axis=1)

mp_hands=mp.solutions.hands
hands=mp_hands.Hands(static_image_mode=True,max_num_hands=2,min_detection_confidence=0.5)
data=[]

def preprocess_landmarks(landmarks):
    landmarks=np.array([[lm.x,lm.y,lm.z] for lm in landmarks]).flatten()
    landmarks=(landmarks-np.mean(landmarks))/np.std(landmarks)
    return landmarks

labels_dict={'food':0,'forget':1,'hello':2,'know':3,'no':4,'say':5,'sky':6,'thanks':7,'yes':8}

for index,row in df.iterrows():
    image_path=row['Gestures']
    label=row['Labels']

    image=cv2.imread(image_path)
    if image is None:
        print(f'Failed to load the image {image_path}')
        continue

    image_rgb=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    results=hands.process(image_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            preprocessed_landmarks=preprocess_landmarks(hand_landmarks.landmark)
            data.append((preprocessed_landmarks,labels_dict[label]))
print(data)
joblib.dump(data, 'ASL_data.pkl')

print("Data processing complete.")

c:\Users\Ankita\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[(array([ 0.45882985,  1.31569882, -1.27340013,  0.36442163,  1.12325713,
       -1.29417666,  0.40240175,  0.90295993, -1.31888317,  0.50636757,
        0.75510117, -1.32889388,  0.58861783,  0.6811885 , -1.33685134,
        0.48374703,  0.9399698 , -1.4318594 ,  0.58354525,  0.63760545,
       -1.47080717,  0.62157676,  0.49380432, -1.45986488,  0.63084572,
        0.41549176, -1.43757175,  0.59465005,  1.00790747, -1.42640002,
        0.70163574,  0.66971352, -1.46646241,  0.70972254,  0.5115652 ,
       -1.44051566,  0.6839187 ,  0.43185791, -1.40558902,  0.6813103 ,
        1.05510687, -1.41223815,  0.76474956,  0.73323179, -1.43771178,
        0.75742711,  0.58058122, -1.40490654,  0.72644688,  0.50630402,
       -1.3683561 ,  0.73159483,  1.08765142, -1.39582782,  0.76040775,
        0.85880627, -1.41016381,  0.73717293,  0.72497078, -1.38777112,
        0.70582132,  0.63829117, -1.35802485]), 0), (array([ 0.54028883,  1.31202027, -1.25365578,  0.43809993,  1.13238565,
       -1

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LeakyReLU,BatchNormalization
from tensorflow.keras.optimizers import Adam
import joblib

data=joblib.load('ASL_data.pkl')

sequence_length = 9  
X = np.array([item[0] for item in data]) 
Y = np.array([item[1] for item in data])

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [2]:
model = Sequential()
model.add(Dense(256, input_shape=(63,)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))


# Compile the model with a slightly reduced learning rate
model.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\Ankita\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Ankita\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [5]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model.fit(X_train, Y_train, epochs=250, batch_size=32, validation_data=(X_test, Y_test), callbacks=[reduce_lr, early_stopping])


Epoch 1/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.7036 - loss: 0.8465 - val_accuracy: 0.8800 - val_loss: 0.4331 - learning_rate: 1.5625e-05
Epoch 2/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7111 - loss: 0.8675 - val_accuracy: 0.8800 - val_loss: 0.4352 - learning_rate: 1.5625e-05
Epoch 3/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6944 - loss: 0.9346 - val_accuracy: 0.8800 - val_loss: 0.4327 - learning_rate: 1.5625e-05
Epoch 4/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6863 - loss: 0.8785 - val_accuracy: 0.8800 - val_loss: 0.4328 - learning_rate: 1.5625e-05
Epoch 5/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6249 - loss: 0.8776 - val_accuracy: 0.8800 - val_loss: 0.4346 - learning_rate: 1.5625e-05
Epoch 6/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6138 - loss: 1.0058 - val_accuracy: 0.8800 - val_loss: 0.4340 - learning_rate: 1.5625e-05
Epoch 7/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6510 - loss:

In [6]:
model.save('ASL_model.h5')
model_json=model.to_json()
with open('gesture_model.json','w') as json_file:
    json_file.write(model_json)

print("Model Training Completed")

Model Training Completed
